<a href="https://colab.research.google.com/github/format81/GenAI-STIX2.1-Generator/blob/main/GenAI_Stix2_1_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#GenAI STIX 2.1 Generator

__Notebook Version:__ 0.1 <br>
__Notebook Author:__ [Antonio Formato](https://www.linkedin.com/in/antonioformato/)<br>


__Python Version:__ >=Python 3.8<br>

__Python Packages:__
- requests
- stix2
- openai
- dotenv

## Table of Contents

1. **Web Scraper** for Blog Post Data
2. Definition of the necessary **functions**
3. **STIX Domain Objects** - AI SDO Generator
4. **STIX Domain Objects** - AI SCO Generator
5. **STIX Domain Objects** - AI SRO Generator
6. **STIX Bundle Generator**
7. **STIX viewer**



**Initialization**

Please install required python package listed in the description above (pip install packages).

In [2]:
!pip3 install python-dotenv
!pip3 install stix2
!pip3 install httpx==0.27.2

## 1 **Extracting Insights** - Building a Web Scraper

This section implements a web scraper to extract text content from a specified blog post URL (infosec writeup).

Utilizing the requests library and BeautifulSoup (bs4), the code ensures successful retrieval of data before proceeding with the extraction.


In [3]:
    #Web scraper
    import requests
    from bs4 import BeautifulSoup
    import pandas as pd

    def scrape_text(url):
        # Add user-agent to avoid issue when scrapping most website
        headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}

        # Send a GET request to the URL
        response = requests.get(url, headers=headers)

        # If the GET request is successful, the status code will be 200
        if response.status_code == 200:
            # Get the content of the response
            page_content = response.content
            # Create a BeautifulSoup object and specify the parser
            soup = BeautifulSoup(page_content, "html.parser")
            # Get the text of the soup object
            text = soup.get_text()
            # Return the text
            return text
        else:
            return "Failed to scrape the website"

    # Enter site
    print("Enter blog post writeup url and press Enter")
    url = input()

    text = scrape_text(url)

Enter blog post writeup url and press Enter
https://cloud.google.com/blog/topics/threat-intelligence/untangling-iran-apt42-operations/


## 2 Definition of the necessary functions

Below is the definition of the prerequisite functions for the notebook:


*   **add_uuid_to_ids** --> Add UUIDs in 'id' field in STIX object
*   **validate_stix_objects** --> Validate STIX objects against the STIX 2.1 standard




In [4]:
# Add UUIDs to 'id' fields in STIX objects, ensuring each object has a unique identifier.
def add_uuid_to_ids(stix_data):
    """
    Add UUIDs to 'id' fields in STIX objects.
    """
    for item in stix_data:
        if 'id' in item:
            object_type = item['type']
            item['id'] = f"{object_type}--{uuid.uuid4()}"
    return stix_data

#Validate a list of STIX objects against the STIX 2.1 standard, identifying any invalid objects.
def validate_stix_objects(stix_objects):
    """
    Validate STIX objects against the STIX 2.1 standard.
    """
    all_valid = True
    invalid_objects = []
    for obj in stix_objects:
        try:
            # Parse the object to validate against the STIX 2.1 standard
            stix_obj = parse(json.dumps(obj), allow_custom=True)
            print(f"Validation passed for object ID: {obj.get('id')}")
        except exceptions.STIXError as se:
            print(f"STIX parsing error for object ID {obj.get('id')}: {se}")
            invalid_objects.append(obj)
            all_valid = False
        except json.JSONDecodeError as je:
            print(f"JSON parsing error: {je}")
            invalid_objects.append(obj)
            all_valid = False
    return all_valid, invalid_objects

**Environment variables**

In this notebook is used Azure OpenAI istance.

The notebook has been tested with **GPT-4o** in Azure OpenAI.


In [6]:
# Setup Azure OpenAI environment
import os
from openai import AzureOpenAI
from dotenv import load_dotenv
from google.colab import userdata
import json

temperature = 0.7

# Set up Azure OpenAI client
# Store your env variables in Colab Secret configuration
client = AzureOpenAI(
  azure_endpoint = userdata.get('AZURE_OPENAI_ENDPOINT'),
  api_key=userdata.get('AZURE_OPENAI_KEY'),
  api_version="2023-05-15"

)

## 3 STIX Domain Objects - **AI SDO Generator**

STIX 2.1 Domain Objects (SDOs) genearator.

Definition of the prompt for generating STIX 2.1 Domain Objects and **SDO generator**

In [7]:
#STIX Domain Objects prompt
system_prompt_sdo = (
    "You are tasked with creating STIX 2.1 Domain Objects (SDOs) from the provided threat intelligence text."
    "Possible SDOs include: Attack Pattern, Campaign, Course of Action, Identity, Indicator, Intrusion Set, Malware, Observed Data, Report, Threat Actor, Tool, Vulnerability, Infrastructure, Relationship, Sighting, Note, Opinion, Grouping, Incident, Location, Malware Analysis."
    "Create relevant SDOs in JSON format, strictly adhering to the STIX 2.1 specification."
    "Ensure the output is a valid JSON array ([...]) containing only SDOs identified with high confidence."
    "The is_family field indicates whether the malware is a family (if true) or an instance (if false). The values true or false are always enclosed in quotes."
    "For id property write just SDO_type-- following this example: \"id\": \"malware--\""
    "Timestamp must be in ISO 8601 format."
    "Don't use created_by_ref and source_ref"
    "The labels property in malware is used to categorize or tag the malware object with descriptive terms (e.g., \"trojan\", \"backdoor\", \"ransomware\"), Must contain at least one string."
    "threat-actor labels property should be an array of strings representing categories or descriptive terms for the threat actor."
    "Return only the JSON array, without any additional text, commentary, or code block delimiters (e.g., json)."
)

user_prompt_stix = f"Text: {text}"

**STIX 2.1 SDO generator**

In [9]:
# STIX 2.1 SDO Generator
import uuid
import json
from stix2 import parse, exceptions

# SDO Generator
def sdo_stix(text):
    # Define the SYSTEM prompt
    response = client.chat.completions.create(
        model=userdata.get('DEPLOYMENT_NAME'),
        messages=[
            {"role": "system", "content": system_prompt_sdo},
            {"role": "user", "content": user_prompt_stix}
        ],
        temperature=temperature
    )
    return response.choices[0].message.content

def correct_invalid_stix(text, invalid_objects, original_stix):
    """
    Ask the LLM to correct invalid STIX output based on the original text and invalid objects.
    """
    # Create a focused invalid output section
    invalid_output = json.dumps(invalid_objects, indent=4)
    response = client.chat.completions.create(
        model=userdata.get('DEPLOYMENT_NAME'),
        messages=[
            {"role": "system", "content": system_prompt_sdo},
            {
                "role": "user",
                "content": f"""
Correct the following invalid STIX objects based on the original text and ensure consistency with the overall STIX data:

Input text:
{text}

Invalid STIX objects:
{invalid_output}

Existing valid STIX data:
{original_stix}
"""
            }
        ],
        temperature=temperature
    )
    return response.choices[0].message.content

# Generate STIX data and ensure valid output
is_valid = False
stix_sdo = ""

while not is_valid:
    # Generate STIX SDO from the LLM
    stix_sdo = sdo_stix(text)

    try:
        # Parse STIX SDO string to a Python list
        stix_data = json.loads(stix_sdo)
        if not isinstance(stix_data, list):
            raise ValueError("Parsed STIX SDO must be a list of dictionaries.")

        # Add UUIDs to IDs
        stix_data = add_uuid_to_ids(stix_data)

        # Print STIX data after UUIDs have been inserted
        print("\nSTIX data after UUID insertion:")
        print(json.dumps(stix_data, indent=4))

        # Validate STIX objects
        is_valid, invalid_objects = validate_stix_objects(stix_data)

        if not is_valid:
            print("Validation failed. Requesting correction from LLM...")
            stix_sdo = correct_invalid_stix(text, invalid_objects, stix_sdo)
        else:
            print("Validation successful. All objects are valid.")

    except json.JSONDecodeError as e:
        print(f"Error parsing STIX SDO: {e}")
        stix_sdo = correct_invalid_stix(text, [], stix_sdo)
    except ValueError as e:
        print(e)
        stix_sdo = correct_invalid_stix(text, [], stix_sdo)

# Final validated STIX SDO as JSON string
stix_sdo = json.dumps(stix_data, indent=4)
print(stix_sdo)


STIX data after UUID insertion:
[
    {
        "type": "threat-actor",
        "id": "threat-actor--a63b9ab7-7253-432d-98ea-5381207c74af",
        "name": "APT42",
        "description": "APT42 is an Iranian state-sponsored cyber espionage actor targeting Western and Middle Eastern NGOs, media organizations, academia, legal services, and activists. It operates on behalf of the Islamic Revolutionary Guard Corps Intelligence Organization (IRGC-IO).",
        "labels": [
            "state-sponsored",
            "cyber-espionage",
            "Iranian"
        ],
        "created": "2024-05-01T00:00:00Z",
        "modified": "2024-05-01T00:00:00Z"
    },
    {
        "type": "intrusion-set",
        "id": "intrusion-set--7e9242ea-dcab-49cf-8e6b-22919d2b361c",
        "name": "APT42 Operations",
        "description": "APT42 uses enhanced social engineering schemes to gain access to victim networks, including cloud environments, by harvesting credentials and using them to gain initial 

## 4 STIX Cyber-Observable Objects - **AI SCO Generator**

STIX 2.1 Cyber-Observable Objects (SCOs) generator.

Definition of the prompt for generating STIX 2.1 Cyber-Observable Objects and **SCO generator**

In [10]:
#STIX Cyber-observable Object prompt
system_prompt_sco = (
    "You are tasked with creating STIX 2.1 Cyber-observable Objects (SCOs) based on the provided threat intelligence write-up."
    "SCOs include: Artifact, Autonomous System, Directory, Domain Name, Email Address, Email Message, File, IPv4 Address, IPv6 Address, MAC Address, Mutex, Network Traffic, Process, Software, URL, User Account, Windows Registry Key, X.509 Certificate, HTTP Request, ICMP, Socket Ext, TCP Ext, Archive Ext, Raster Image Ext, NTFS Ext, PDF Ext, UNIX Account Ext, Windows PE Binary Ext, Windows Process Ext, Windows Service Ext, Windows Registry Ext, JPEG File Ext, Email MIME Component, Email MIME Multipart Type, Email MIME Message Type, Email MIME Text Type."
    "Create relevant STIX 2.1 SCOs in JSON format based on the information provided in the text."
    "Strictly follow the STIX 2.1 specification, ensuring no properties are used that are not defined in the specification"
    "Ensure the JSON output is valid, starting with [ and closing with ]."
    "STIX SCO objects require at least type, id and value properties"
    "Only provide output if one or more SCOs can be identified with reasonable certainty from the text."
    "Ensure the structure and format are fully compliant with STIX 2.1."
    "id STIX identifier must match <object-type>--<UUID>"
    "Return only the JSON array, without any additional text, commentary, or code block delimiters (e.g., json)."
  )

**STIX 2.1 SCO generator**

In [11]:
# STIX 2.1 SCO Generator
import uuid
import json
from stix2 import parse, exceptions

# SCO Generator
def sco_stix(text):
    """
    Generate STIX SCOs using the LLM.
    """
    response = client.chat.completions.create(
        model=userdata.get('DEPLOYMENT_NAME'),
        messages=[
            {"role": "system", "content": system_prompt_sco},
            {"role": "user", "content": user_prompt_stix}
        ],
        temperature=temperature
    )
    return response.choices[0].message.content

def correct_invalid_stix(text, invalid_objects, original_stix):
    """
    Ask the LLM to correct invalid STIX output based on the original text and invalid objects.
    """
    # Create a focused invalid output section
    invalid_output = json.dumps(invalid_objects, indent=4)
    response = client.chat.completions.create(
        model=userdata.get('DEPLOYMENT_NAME'),
        messages=[
            {"role": "system", "content": system_prompt_sco},
            {
                "role": "user",
                "content": f"""
Correct the following invalid STIX objects based on the original text and ensure consistency with the overall STIX data:

Input text:
{text}

Invalid STIX objects:
{invalid_output}

Existing valid STIX data:
{original_stix}
"""
            }
        ],
        temperature=temperature
    )
    return response.choices[0].message.content

# Generate and validate STIX SCO
is_valid = False
stix_sco = ""

while not is_valid:
    # Generate STIX SCO from the LLM
    stix_sco = sco_stix(text)

    try:
        # Parse STIX SCO string to a Python list
        stix_data = json.loads(stix_sco)
        if not isinstance(stix_data, list):
            raise ValueError("Parsed STIX SCO must be a list of dictionaries.")

        # Add UUIDs to IDs
        stix_data = add_uuid_to_ids(stix_data)

        # Print STIX data after UUIDs have been inserted
        print("\nSTIX SCO data after UUID insertion:")
        print(json.dumps(stix_data, indent=4))

        # Validate STIX objects
        is_valid, invalid_objects = validate_stix_objects(stix_data)

        if not is_valid:
            print("Validation failed. Requesting correction from LLM...")
            stix_sco = correct_invalid_stix(text, invalid_objects, stix_sco)
        else:
            print("Validation successful. All objects are valid.")

    except json.JSONDecodeError as e:
        print(f"Error parsing STIX SCO: {e}")
        stix_sco = correct_invalid_stix(text, [], stix_sco)
    except ValueError as e:
        print(e)
        stix_sco = correct_invalid_stix(text, [], stix_sco)

# Final validated STIX SCO as JSON string
stix_sco = json.dumps(stix_data, indent=4)
print(stix_sco)


STIX SCO data after UUID insertion:
[
    {
        "type": "domain-name",
        "id": "domain-name--5f2730f5-a17a-4842-9744-73aacfcefcdb",
        "value": "washinqtonpost.press"
    },
    {
        "type": "domain-name",
        "id": "domain-name--31e83530-c412-47f0-a636-f35293916b52",
        "value": "ksview.top"
    },
    {
        "type": "domain-name",
        "id": "domain-name--dde332d5-e63a-4796-965b-39e84bbf7f3a",
        "value": "honest-halcyon-fresher.buzz"
    },
    {
        "type": "domain-name",
        "id": "domain-name--ca21e8b0-0f63-4747-8039-5ebf7e9f17e1",
        "value": "n9.cl"
    },
    {
        "type": "domain-name",
        "id": "domain-name--8fdd479b-587b-434c-95f5-499165203aac",
        "value": "review.modification-check.online"
    },
    {
        "type": "domain-name",
        "id": "domain-name--1b837c95-98c9-4316-bd60-da67f956dfe9",
        "value": "nterview.site"
    },
    {
        "type": "domain-name",
        "id": "domain-name--332

## 5 STIX Relationship Objects - **AI SRO Generator**

STIX 2.1 Relationship Objects (SROs) generator.

Definition of the prompt for generating STIX 2.1 Relationship Objects and **SRO generator**


In [12]:
#STIX Relationship Object prompt
system_prompt_sro = (
    "You are tasked with creating a STIX 2.1 Relationship Object (SRO) based on the provided writeup about threat intelligence text SDOs and SCOs"
    "Remember a relationship is a link between STIX Domain Objects (SDOs), STIX Cyber-observable Objects (SCOs), or between an SDO and a SCO that describes the way in which the objects are related. Relationships can be represented using an external STIX Relationship Object (SRO) or, in some cases, through certain properties which store an identifier reference that comprises an embedded relationship, (for example the created_by_ref property)."
    "Create STIX Objects, in json format."
    "Identify Relationships: For each entity (like intrusion-set, malware, infrastructure, domain-name, file, directory), identify how they relate to each other. For example, malware might use infrastructure for command and control, or an intrusion set might leverage certain domains"
    "Use relationship Objects: Use relationship objects to connect entities. This object will specify the source and target entities and define the nature of the relationship (e.g., \"uses\", \"communicates with\")"
    "Ensure Consistent Referencing: Make sure that every object you want to relate is referenced correctly using their id in the relationship objects."
    "Pay attention to properties, don't use properties not defined in STIX 2.1 specification"
    "Start with [ and close with ] , no other content before [ and after ]"
    "If you cannot identify a specific SCO from the provided text, simply do not do anything."
    "Provide output only if you can identify one or more SCOs with reasonable certainty."
    "Pay attention to provide valid json."
    "Pay attention to provide valid STIX 2.1 structure."
    "Return only the JSON array, without any additional text, commentary, or code block delimiters (e.g., json)."
  )

user_stix_sdo_sco_text = f"Text of writeup: {text},  {stix_sdo} , {stix_sco}"

**STIX 2.1 SRO generator**

In [13]:
# STIX 2.1 SRO generator
import uuid
import json
from stix2 import parse, exceptions

# SRO Generator
def sro_stix(text, stix_sdo, stix_sco):
    """
    Generate STIX SROs using the LLM.
    """
    response = client.chat.completions.create(
        model=userdata.get('DEPLOYMENT_NAME'),
        messages=[
            {"role": "system", "content": system_prompt_sro},
            {
                "role": "user",
                "content": f"""
Generate STIX 2.1 relationship objects (SROs) based on the following:

Input text:
{text}

SDO:
{stix_sdo}

SCO:
{stix_sco}
"""
            }
        ],
        temperature=temperature
    )
    return response.choices[0].message.content

def correct_invalid_stix(text, invalid_objects, original_stix):
    """
    Ask the LLM to correct invalid STIX output based on the original text and invalid objects.
    """
    # Create a focused invalid output section
    invalid_output = json.dumps(invalid_objects, indent=4)
    response = client.chat.completions.create(
        model=userdata.get('DEPLOYMENT_NAME'),
        messages=[
            {"role": "system", "content": system_prompt_sro},
            {
                "role": "user",
                "content": f"""
Correct the following invalid STIX relationship objects (SROs) based on the original text and ensure consistency with the SDO and SCO data:

Input text:
{text}

Invalid SRO objects:
{invalid_output}

SDO:
{stix_sdo}

SCO:
{stix_sco}
"""
            }
        ],
        temperature=temperature
    )
    return response.choices[0].message.content

# Generate and validate STIX SRO
is_valid = False
stix_sro = ""

while not is_valid:
    # Generate STIX SRO from the LLM
    stix_sro = sro_stix(text, stix_sdo, stix_sco)

    try:
        # Parse STIX SRO string to a Python list
        stix_data = json.loads(stix_sro)
        if not isinstance(stix_data, list):
            raise ValueError("Parsed STIX SRO must be a list of dictionaries.")

        # Add UUIDs to IDs
        stix_data = add_uuid_to_ids(stix_data)

        # Print STIX data after UUIDs have been inserted
        print("\nSTIX SRO data after UUID insertion:")
        print(json.dumps(stix_data, indent=4))

        # Validate STIX objects
        is_valid, invalid_objects = validate_stix_objects(stix_data)

        if not is_valid:
            print("Validation failed. Requesting correction from LLM...")
            stix_sro = correct_invalid_stix(text, invalid_objects, stix_sro)
        else:
            print("Validation successful. All objects are valid.")

    except json.JSONDecodeError as e:
        print(f"Error parsing STIX SRO: {e}")
        stix_sro = correct_invalid_stix(text, [], stix_sro)
    except ValueError as e:
        print(e)
        stix_sro = correct_invalid_stix(text, [], stix_sro)

# Final validated STIX SRO as JSON string
stix_sro = json.dumps(stix_data, indent=4)
print(stix_sro)


STIX SRO data after UUID insertion:
[
    {
        "type": "relationship",
        "id": "relationship--8624e252-244b-43b2-828b-4860687979cd",
        "relationship_type": "uses",
        "source_ref": "threat-actor--a63b9ab7-7253-432d-98ea-5381207c74af",
        "target_ref": "intrusion-set--7e9242ea-dcab-49cf-8e6b-22919d2b361c",
        "created": "2024-05-01T00:00:00Z",
        "modified": "2024-05-01T00:00:00Z"
    },
    {
        "type": "relationship",
        "id": "relationship--6a653878-988c-49fe-9463-7b5a91541d9e",
        "relationship_type": "uses",
        "source_ref": "intrusion-set--7e9242ea-dcab-49cf-8e6b-22919d2b361c",
        "target_ref": "malware--d10904bd-10b9-4045-b514-f6c5f7d1de50",
        "created": "2024-05-01T00:00:00Z",
        "modified": "2024-05-01T00:00:00Z"
    },
    {
        "type": "relationship",
        "id": "relationship--b0dd9712-f460-422a-92d7-9d229f645a03",
        "relationship_type": "uses",
        "source_ref": "intrusion-set--7e9242e

## 6 **STIX Bundle Generator**

STIX 2.1 bundle genearator.

Let's combine SDO, SCO, and SRO to generate the **STIX 2.1 bundle**.

In [14]:
#Create STIX 2.1 Bundle
import json
from stix2 import Bundle
import uuid
from datetime import datetime

def remove_brackets(text):
    """
    Remove leading '[' and trailing ']' and format inner objects into a valid JSON array.
    """
    if not text.startswith('[') or not text.endswith(']'):
        raise ValueError("Input text does not start with '[' or end with ']'")

    # Remove brackets and split into individual JSON objects
    objects_str = text[1:-1].strip()  # Remove brackets and leading/trailing whitespace
    objects_list = objects_str.split('}{')

    # Add back the missing curly braces and format into a valid JSON array
    formatted_objects = "[" + ", ".join(["{" + obj + "}" if not obj.startswith("{") and not obj.endswith("}") else obj for obj in objects_list]) + "]"

    return formatted_objects

def create_stix_bundle(sdo_data, sco_data, sro_data):
    """
    Create a STIX 2.1 bundle from input SDO, SCO, and SRO data.

    Args:
        sdo_data (list): List of valid SDO objects.
        sco_data (list): List of valid SCO objects.
        sro_data (list): List of valid SRO objects.

    Returns:
        str: A STIX 2.1 bundle in JSON format.
    """
    # Combine SDO, SCO, and SRO data
    all_objects = sdo_data + sco_data + sro_data
    #print("all_objects:", all_objects)

    bundle = {
        "type": "bundle",
        "id": f"bundle--{uuid.uuid4()}",
        "objects": all_objects
    }

    # Return the serialized bundle
    return json.dumps(bundle, indent=4)

# Remove brackets and parse as JSON
stix_sdo_objects = json.loads(remove_brackets(stix_sdo))
stix_sco_objects = json.loads(remove_brackets(stix_sco))
stix_sro_objects = json.loads(remove_brackets(stix_sro))

# Create STIX bundle
stix_bundle = create_stix_bundle(stix_sdo_objects, stix_sco_objects, stix_sro_objects)

# Output the result
print("STIX Bundle:")
print(stix_bundle)
# Generate the filename
filename = f"bundle_{datetime.now().strftime('%Y_%m_%d_%H_%M')}.json"
# Write the STIX bundle to the file
with open(filename, 'w') as file:
    file.write(stix_bundle)

print(f"STIX bundle written to file: {filename}")

STIX Bundle:
{
    "type": "bundle",
    "id": "bundle--00914181-1d69-4d79-9f94-e8e1b472505a",
    "objects": [
        {
            "type": "threat-actor",
            "id": "threat-actor--a63b9ab7-7253-432d-98ea-5381207c74af",
            "name": "APT42",
            "description": "APT42 is an Iranian state-sponsored cyber espionage actor targeting Western and Middle Eastern NGOs, media organizations, academia, legal services, and activists. It operates on behalf of the Islamic Revolutionary Guard Corps Intelligence Organization (IRGC-IO).",
            "labels": [
                "state-sponsored",
                "cyber-espionage",
                "Iranian"
            ],
            "created": "2024-05-01T00:00:00Z",
            "modified": "2024-05-01T00:00:00Z"
        },
        {
            "type": "intrusion-set",
            "id": "intrusion-set--7e9242ea-dcab-49cf-8e6b-22919d2b361c",
            "name": "APT42 Operations",
            "description": "APT42 uses enhanced 

## 7 **STIX Viewer**

STIX 2.1 bundle Visualizer.

To visualize the STIX bundle, we use the cti-stix-visualization project, inserted as an iFrame in the notebook.

[STIX Visualizer](https://oasis-open.github.io/cti-stix-visualization/)


In [ ]:
# Visualize STIX Bundle
# cut & paste json file
from IPython.display import IFrame

IFrame(src='https://oasis-open.github.io/cti-stix-visualization/', width=1200, height=1000)